### Get Packages 

In [35]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import datetime

%run ml_helpers.ipynb
%run ml.ipynb

         Unnamed: 0  Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  \
1325733     1325733  2234  2019     12  Second  0.00   0.0    53    32   
1209162     1209162  1934  2019      7   Third  0.01   0.0    90    73   
1223045     1223045  1113  2019      8  Second  0.00   0.0    80    60   
1071197     1071197   922  2019      1   First  0.00   0.1    36    27   
1268945     1268945   334  2019      9  Second  0.00   0.0    87    66   
...             ...   ...   ...    ...     ...   ...   ...   ...   ...   
1072555     1072555   735  2019      1   Third  0.01   0.1    32    27   
1120900     1120900  1522  2019      3  Second  0.00   0.0    50    30   
1138451     1138451   334  2019      4  Second  0.00   0.0    54    37   
1143525     1143525   434  2019      4   Third  0.00   0.0    65    41   
1268116     1268116  1511  2019      9   First  0.99   0.0    69    62   

         count_l_stops  ...  count_restaurants  count_bars  count_daycares  \
1325733            0.0  ...      

Finished one-hot encoding...
Finished standardizing...
Working on: [2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [30]:
#Get Final data: 
data = pd.read_csv("../intermediate_data/high_crime_labeled.csv")
#data = data.drop(columns=["beat", "beat_num_x", "Crimes", "Arrest", "Domestic", "Serious"])

data = convert_to_categorical(data, ["district", "sector",
                                    "Month",
                                     "Watch", "Beat"])
data.drop(columns = ["Unnamed: 0"], inplace=True)
data_list_20 = prep_data_test(data, "high_crime",
                                        2, "Year", ["district", "sector", 
                                                    "Month", "Beat", "Watch"])


       beat  beat_num district sector  Year Month   Watch  Beat  Crimes  \
29557     1      1713       17      1  2018     1   First  1713       8   
29558     1      1713       17      1  2018     1  Second  1713      16   
29559     1      1713       17      1  2018     1   Third  1713      24   
29560     1      1713       17      1  2018     2   Third  1713      16   
29561     1      1713       17      1  2018     2   First  1713       8   
...     ...       ...      ...    ...   ...   ...     ...   ...     ...   
49256     1       312        3      1  2019    11  Second   312      43   
49257     1       312        3      1  2019    11   Third   312      28   
49258     1       312        3      1  2019    12  Second   312      33   
49259     1       312        3      1  2019    12   First   312      19   
49260     1       312        3      1  2019    12   Third   312      31   

       Serious  ...  count_restaurants  count_bars  count_daycares  \
29557      1.0  ...          

Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2017 2018]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [31]:
def compute_scores_final_data(grid, model, model_name, data_list, target, target_year):
    '''
    Function that computes dataframe for results of best models
    Inputs:
        grid (dict): maps the model_name to a list which contains a dictionary mapping parameter (keys) to their 
                     values in the gest model
        model (sklearn): An sklearn algorithm
        model_name: a string assigned to the model, typically the same as model without the last parentheses
        data_list: list of tuples of dataframes. IN THIS CASE, this should include for 2020
        target (string): the column we seek to predict in the data frames
        target_year (int): 2020, the year we seek to predict
    Output:
        results (df): a dataframe with the results
    
    Note: the code that removes the params columns comes from here: https://stackoverflow.com/questions/19071199/drop-columns-whose-name-contains-a-specific-string-from-pandas-dataframe
    '''
    results = results_by_year(model, 
                                    model_name, grid, 
                                    data_list, 
                                    target, 
                                    target_year)
    results["model_type"]= model_name
    results["params"]=results["parameters_2020"]
    
    bool_array = results.columns.str.startswith("parameters")
    results2 = results.loc[:,~bool_array]
    return results2

## Logistic Regression

In [32]:
#Logistic Regression!
grid_log_reg ={"LogisticRegression": [{'penalty': 'l2',
                              'C': 0.001,
                              'random_state': 0}]}

In [33]:
results_log = compute_scores_final_data(grid_log_reg, LogisticRegression(),
                                    "LogisticRegression", data_list_20, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]
precision is: (0.9761904761904762,) |recall is: (0.3734989648033126,)


ValueError: 3 columns passed, passed data had 2 columns

## KNN

In [ ]:
#KNN!
grid_knn = {"KNeighborsClassifier": [{'weights': 'uniform', 'n_neighbors': 10}]}
results_knn = compute_scores_final_data(grid_knn, KNeighborsClassifier(),
                                    "KNeighborsClassifier", data_list_20, "high_crime", 2020)

In [ ]:
results_knn_to_plot = results_knn[["precision_2017", "precision_2018", "precision_2019", "precision_2020"]]

r = results_knn_to_plot.transpose().reset_index()
r

In [ ]:
sns.countplot(x=r["index"])


## SVM: 

In [ ]:
grid_svm = { :[{}]}
results_svm = compute_scores_final_data()

## Naive Bayes: 

In [ ]:
grid_nb = { :[{}]}
results_nb = compute_scores_final_data()

## Decision Tree:

In [ ]:
grid_dt = { :[{}]}
results_dt = compute_scores_final_data()

## Random Forest:

In [ ]:
grid_rf = { :[{}]}
results_rf = compute_scores_final_data()

## Aggregate Files:

In [ ]:
results = pd.concat([results_log, results_knn
#                    , results_svm, results_nb, results_dt, results_rf
                    ])
results